# Asignación y normalización de labels

En este notebook se procederá:
1. En primer lugar, a asignar las labels -sin normalizar- a las distintas imágenes.
2. Una vez las labels han sido correctametne asignadas, procederán a normalizarse.

In [1]:
import os
import pandas as pd
from tqdm import tqdm  # Para mostrar progreso
import rasterio


## Consultamos las dimensiones de la imagen que vayamos a recortar

In [2]:
# Selección de la imagen
NUM_TILE = 62
image_path  = 'G:\\.shortcut-targets-by-id\\1pYgV5EIk4-LapLNhlCwpQaDAzuqNffXG\\doctorado_albert\\conteo_pinguinos\\recortes'
image_name = f"recorte_{NUM_TILE}.tif"
full_image_path = os.path.join(image_path, image_name)

# Abrir la imagen con rasterio
with rasterio.open(full_image_path) as dataset:
    # Obtener el ancho (número de columnas) y alto (número de filas) de la imagen
    width = dataset.width
    height = dataset.height

# Mostrar el resultado
print(f'Ancho: {width} píxeles')
print(f'Alto: {height} píxeles')

# Ancho: 10195 píxeles
# Alto: 11420 píxeles

Ancho: 10195 píxeles
Alto: 11420 píxeles


In [3]:


def generar_txt_yolo(
    image_dir: str, 
    csv_file: str = 'coords/coords.csv',  # Asegúrate de que este archivo tenga las coordenadas no normalizadas
    output_dir: str = 'coords/labels'  # Directorio donde se guardarán los archivos .txt generados
) -> None:
    """
    Genera archivos de etiquetas YOLO para cada tile basado en coordenadas no normalizadas.

    Args:
    - csv_file (str): Ruta al archivo CSV con las coordenadas no normalizadas.
    - image_dir (str): Directorio donde están las imágenes recortadas.
    - output_dir (str): Directorio donde se guardarán los archivos .txt generados.
    """
    # Cargar el CSV con las coordenadas no normalizadas
    data = pd.read_csv(csv_file, header=None, names=['class', 'x_center', 'y_center', 'width', 'height'])

    # Convertir las columnas a tipos numéricos
    data = data.drop(index=0)

    data['x_center'] = pd.to_numeric(data['x_center'])
    data['y_center'] = pd.to_numeric(data['y_center'])
    data['width'] = pd.to_numeric(data['width'])
    data['height'] = pd.to_numeric(data['height'])

    # Asegurarse de que el directorio de salida existe
    os.makedirs(output_dir, exist_ok=True)
    
    # Obtener los nombres de las imágenes (tiles)
    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.tif', '.png', '.tiff'))]

    for image_file in tqdm(image_files, desc="Generando archivos .txt"):
        image_path = os.path.join(image_dir, image_file)
        
        # Abrimos la imagen georeferenciada con rasterio
        with rasterio.open(image_path) as src:
            # Obtener las coordenadas geográficas de la imagen (bounding box)
            xmin, ymin, xmax, ymax = src.bounds  # Estos son los límites geográficos de la imagen

        # Filtrar las coordenadas que caen dentro de las coordenadas geográficas de la imagen
        filtered_data = data[
            (data['x_center'] >= xmin) & (data['x_center'] <= xmax) &
            (data['y_center'] >= ymin) & (data['y_center'] <= ymax)
        ]

        # Crear el archivo .txt para este tile
        if not filtered_data.empty:
            # Crear el archivo .txt para este tile
            output_path = os.path.join(output_dir, f"{os.path.splitext(image_file)[0]}.txt")
            filtered_data[['class', 'x_center', 'y_center', 'width', 'height']].to_csv(
                output_path, sep=' ', index=False, header=False
            )

    print(f"Archivos .txt generados en {output_dir}")




In [4]:
# Ejemplo de uso
csv_file = "coords/yolo_coords.csv"  # Archivo CSV con coordenadas no normalizadas
image_dir = f"cut_tiles/tiles_500x500_{NUM_TILE}"  # Carpeta con imágenes de 500x500
output_dir = "coords/labels_sin_normalizar"  # Carpeta donde se guardarán los .txt

generar_txt_yolo(image_dir=image_dir, csv_file=csv_file, output_dir=output_dir)

Generando archivos .txt: 100%|██████████| 400/400 [00:06<00:00, 60.71it/s]

Archivos .txt generados en coords/labels_sin_normalizar


## Normalizamos las labels

Para cara archivo en txt, será necesario normalizarlo para que pueda ser leido por yolo. Los archivos normalizados pasarán a la carpeta labels_normalized. Una vez estén en esta caqrpeta, podremos pasar al paso número 5 (jungar cada label con su imagen correspondiente dentro de la carpeta dataset).

In [6]:
def normalize_yolo_coords(
    tiff_file: str,  # ruta de la imagen TIFF
    txt_file_coords: str, 
    output_file: str,
) -> None:
    """
    Normaliza todas las coordenadas del dataset completo al formato YOLO.
    
    Args:
    - txt_file_coords: str - Ruta al archivo TXT con todas las coordenadas en formato YOLO (Id, x_center, y_center, width, height).
    - tiff_file: str - Ruta al archivo TIFF que define el área de referencia. Se obtienen así los límites.
    - output_file: str - Nombre del archivo de salida con las coordenadas normalizadas.
    """
    # Cargar datos del archivo TXT
    data = pd.read_csv(txt_file_coords, delimiter=' ', encoding='ISO-8859-1', header=None, names=['class', 'x_center', 'y_center', 'width', 'height'])

    # Abrir el archivo TIFF para obtener metadatos
    with rasterio.open(tiff_file) as src:
        transform = src.transform
        width, height = src.width, src.height

        # Definir los límites del área total
        top_left = transform * (0, 0)
        bottom_right = transform * (src.width, src.height)
        min_x, max_y = top_left
        max_x, min_y = bottom_right

    # Filtrar los puntos dentro del área total (opcional, por si hay datos fuera del rango del TIFF)
    data = data[
        (data['x_center'] >= min_x) & (data['x_center'] <= max_x) &
        (data['y_center'] >= min_y) & (data['y_center'] <= max_y)
    ]

    # Normalizar las coordenadas y dimensiones
    data['x_center'] = (data['x_center'] - min_x) / (max_x - min_x)  # Normalización en X
    data['y_center'] = (data['y_center'] - min_y) / (max_y - min_y)  # Normalización en Y
    data['width'] = data['width'] / width  # Normalización del ancho
    data['height'] = data['height'] / height  # Normalización de la altura

    # Guardar el dataset en formato YOLO
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    data.to_csv(output_file, index=False, header=False, sep=' ')



In [7]:
coords_sin_normalizar = "coords/labels_sin_normalizar"
coords_normalized = "coords/labels_normalized"


for file in os.listdir(coords_sin_normalizar):
    coords_file = os.path.join(coords_sin_normalizar, file)
    output_file = os.path.join(coords_normalized, file)
    
    normalize_yolo_coords(
        tiff_file=full_image_path,
        txt_file_coords=coords_file, 
        output_file=output_file, 
    )